<a href="https://colab.research.google.com/github/sadikul1500/Movie-Recommender/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
import pandas as pd

In [ ]:
print('select movies data file')
uploaded = files.upload()

select movies data file


Saving movies.csv to movies.csv


In [ ]:
for f in uploaded.keys():
  movies = pd.read_csv(f, usecols=['movieId', 'title'], dtype={'title':'str'})
  print(movies.head())


   movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [ ]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB
None


In [ ]:
print('select user ratings data file')
uploaded = files.upload()

select user ratings data file


Saving ratings.csv to ratings.csv


In [ ]:
for f in uploaded.keys():
  ratings = pd.read_csv(f, usecols=['userId', 'movieId', 'rating'])

In [ ]:
print(ratings.head())
print(ratings.info())

   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB
None


In [ ]:
array_ratings = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(1e-8)
print(array_ratings.head())
#print(array_ratings[0])

userId            1             2    ...           609           610
movieId                              ...                            
1        4.000000e+00  1.000000e-08  ...  3.000000e+00  5.000000e+00
2        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08
3        4.000000e+00  1.000000e-08  ...  1.000000e-08  1.000000e-08
4        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08
5        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08

[5 rows x 610 columns]


In [ ]:
print(type(array_ratings))
array_ratings_np = array_ratings.to_numpy()
print(len(array_ratings_np))

<class 'pandas.core.frame.DataFrame'>
9724


In [ ]:
#don't need 
from scipy.sparse import csr_matrix
matrix_ratings = csr_matrix(array_ratings.values)
print(type(matrix_ratings))
print(matrix_ratings) 

In [ ]:
print(movies.loc[90]['title'])

Mr. Wrong (1996)


In [ ]:
#knn
from random import shuffle

def getDistance(test, train):
    test = test[:len(test)-1]
    distance = []

    #j = 0
    for training in train:
        x = 0

        for i in range(len(test)):
            x += (float(test[i]) - float(training[i])) ** 2
        #print(training)
        distance.append([x ** .5, moviestraining[-1]])

    return distance


def getPrediction(distance, k):
    output_values = [dist[-1] for dist in distance[:k]]
    prediction = max(set(output_values), key=output_values.count)
    return prediction


def weightedKnn(distance, k):
    output_values = [dist[-1] for dist in distance[:k]]
    output_values = set(output_values)
    output_values = list(output_values)

    values = set(map(lambda x: x[1], distance))
    new_list = [[y[0] for y in distance if y[1] == x] for x in values]

    weight = []
    myWeight = []

    for i in range((len(output_values))):
        sum = 0.0
        wSum = 0.0
        for j in new_list[i]:
            sum += 1.0 / (j + 1e-10)
            wSum += 1.0 / (j**2 + 1e-10)

        weight.append(sum)
        myWeight.append(len(new_list[i]) * sum)

    index = weight.index(max(weight))
    wIndex = myWeight.index(max(myWeight))

    return output_values[index], output_values[wIndex]


def writeInFile(List, wList, mList):
    for list in List:
        oFile.write(str(list) + ',')

    oFile.write('\n')

    for list in wList:
        woFile.write(str(list) + ',')

    woFile.write('\n')

    for list in mList:
        moFile.write(str(list) + ',')

    moFile.write('\n')


def evaluateAlgorithm(train, test, k, oFile, woFile, moFile):
    #distance = []
    count = 0
    wCount = 0
    mCount = 0
    for i in range(len(test)):
        test_values = test[i]
        distance = getDistance(test_values, train)
        distance.sort()

        prediction = getPrediction(distance, k)
        mPrediction, wPrediction = weightedKnn(distance, k)
        #oFile.write([str(x) + ',' for x in test_values] + ',' + prediction + '\n')
        List = [test_values, prediction]
        wList = [test_values, wPrediction]
        mList = [test_values, mPrediction]
        writeInFile(List, wList, mList)




        print(prediction + ' ' + test_values[-1])

        if prediction == test_values[-1]:
            count += 1

        if wPrediction == test_values[-1]:
            wCount += 1

        if mPrediction == test_values[-1]:
            mCount += 1

    oFile.write('accuracy : ' + str(count / len(test)) + '\n\n')
    woFile.write('accuracy : ' + str(wCount / len(test)) + '\n\n')
    moFile.write('accuracy : ' + str(mCount / len(test)) + '\n\n')
    print('\n')

    return count / len(test), wCount / len(test), mCount / len(test)


if __name__ == '__main__':
    fileName = 'G:\\5 th semester\\dbms2\\knn\\iris.txt'
    oFile = open('outIris.txt', 'w')
    woFile = open('weight.txt', 'w')
    moFile = open('mWeight.txt', 'w')

    file = open(fileName, 'r')
    lines = file.readlines()
    numberOfLines = len(lines)
    print(numberOfLines)

    data = []
    i = 0
    for line in lines:
        values = line.split(',')
        data.append(values)
        i += 1

    shuffle(data)
    shuffledData = list.copy(data)
    #print(data)

    numberOfFold = int(input("number of fold: "))
    k = int(input('value of k: '))
    percent = int(len(shuffledData) / numberOfFold)
    accuracy = []
    wAccuracy = []
    mAccuracy = []

    for i in range(numberOfFold):
        start = i * percent
        end = start + percent
        test = shuffledData[start : end]
        train = shuffledData[: start] + shuffledData[end-1:]
        #print(test)
        #print(train)
        neutral, weight, mWeight = evaluateAlgorithm(train, test, k, oFile, woFile, moFile)
        accuracy.append(neutral)
        wAccuracy.append(weight)
        mAccuracy.append(mWeight)

    print(accuracy)
    meanAccuracy = sum(accuracy) / len(accuracy)
    print('mean accuracy : ', meanAccuracy)

    print(wAccuracy)
    w_meanAccuracy = sum(wAccuracy) / len(wAccuracy)
    print('weighted mean accuracy : ', w_meanAccuracy)

    print(mAccuracy)
    m_meanAccuracy = sum(mAccuracy) / len(mAccuracy)
    print('weighted mean accuracy : ', m_meanAccuracy)

    file.close()
    oFile.close()
    woFile.close()
    moFile.close()

In [ ]:
def getPrediction(distance, k):
    output_values = [dist[-1] for dist in distance[:k]]
    #prediction = max(set(output_values), key=output_values.count)
    return output_values #prediction

In [ ]:
print(array_ratings_np[0][0])

4.0


In [ ]:
def getDistance(movieIndex, train): #, train
    #test = test[:len(test)-1]
    distance = []

    j = 0
    for training in train:
        x = 0
        for i in range(len(training)):
            x += (float(train[movieIndex][i]) - float(training[i])) ** 2
        #print(training)
        movie = movies.loc[j]['title'] #movies[movies['movieId'] == j+1].iloc[0]['title']
        distance.append([x ** .5, movie])
        j += 1

    return distance


In [ ]:
def evaluateAlgorithm(test, k, movieIndex):
    #distance = []
    #count = 0
    #wCount = 0
    #mCount = 0
    #for i in range(len(array_ratings_np)):
        # = test[i]
    distance = getDistance(movieIndex, array_ratings_np) #, train
    distance.sort()

    prediction = getPrediction(distance, k)
    print(prediction)
    '''
    mPrediction, wPrediction = weightedKnn(distance, k)
    #oFile.write([str(x) + ',' for x in test_values] + ',' + prediction + '\n')
    List = [test_values, prediction]
    wList = [test_values, wPrediction]
    mList = [test_values, mPrediction]
    writeInFile(List, wList, mList)




    print(prediction + ' ' + test_values[-1])

    if prediction == test_values[-1]:
        count += 1

    if wPrediction == test_values[-1]:
        wCount += 1

    if mPrediction == test_values[-1]:
        mCount += 1

    oFile.write('accuracy : ' + str(count / len(test)) + '\n\n')
    woFile.write('accuracy : ' + str(wCount / len(test)) + '\n\n')
    moFile.write('accuracy : ' + str(mCount / len(test)) + '\n\n')
    print('\n')

    return count / len(test), wCount / len(test), mCount / len(test)
    '''

In [ ]:
test = input('enter a movie name with release year (e.g. Toy Story (1995)) ')
k = int(input('expected number of recommendations '))

#movieId = movies[movies['movieId'] == test].iloc[0]['movieid']



enter a movie name with release year (e.g. Toy Story (1995)) Superman (1978)
expected number of recommendations 10


In [1]:
import string
s = string.capwords('The BOSS baby')
print(s)

The Boss Baby


In [ ]:
print(movies.index[movies['title'] == test].tolist()[0])

1986


In [ ]:
movieIndex = movies.index[movies['title'] == test].tolist()[0] #movies[movies['title'] == test].to_numpy()
print(movieIndex)
evaluateAlgorithm(test, k, movieIndex)

1986
['Superman (1978)', 'Superman II (1980)', 'Superman III (1983)', 'Pale Rider (1985)', 'Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988)', 'Virus (1999)', 'Timecop (1994)', 'Madeline (1998)', "National Lampoon's Vacation (1983)", 'Clue (1985)']


In [ ]:
if __name__ == '__main__':
    #fileName = 'G:\\5 th semester\\dbms2\\knn\\iris.txt'
    #oFile = open('outIris.txt', 'w')
    #woFile = open('weight.txt', 'w')
    #moFile = open('mWeight.txt', 'w')

    #file = open(fileName, 'r')
    #lines = file.readlines()
    #numberOfLines = len(lines)
    #print(numberOfLines)
    '''
    data = []
    i = 0
    for line in lines:
        values = line.split(',')
        data.append(values)
        i += 1

    shuffle(data)
    shuffledData = list.copy(data)
    #print(data)

    numberOfFold = int(input("number of fold: "))
    '''
    k = int(input('value of k: '))
    percent = int(len(shuffledData) / numberOfFold)
    accuracy = []
    wAccuracy = []
    mAccuracy = []

    for i in range(numberOfFold):
        start = i * percent
        end = start + percent
        test = shuffledData[start : end]
        train = shuffledData[: start] + shuffledData[end-1:]
        #print(test)
        #print(train)
        neutral, weight, mWeight = evaluateAlgorithm(train, test, k, oFile, woFile, moFile)
        accuracy.append(neutral)
        wAccuracy.append(weight)
        mAccuracy.append(mWeight)

    print(accuracy)
    meanAccuracy = sum(accuracy) / len(accuracy)
    print('mean accuracy : ', meanAccuracy)

    print(wAccuracy)
    w_meanAccuracy = sum(wAccuracy) / len(wAccuracy)
    print('weighted mean accuracy : ', w_meanAccuracy)

    print(mAccuracy)
    m_meanAccuracy = sum(mAccuracy) / len(mAccuracy)
    print('weighted mean accuracy : ', m_meanAccuracy)

    file.close()
    oFile.close()
    woFile.close()
    moFile.close()